# Продвинутый Python, лекция 3

**Лектор:** Петров Тимур

**Семинаристы:** Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

Сегодня говорим про тестирование и логгирование (на примере TDD)

Любая программа, которую вы делаете, надо тестировать. К сожалению, в реальной жизни у вас нет сразу тестов, так что и писать их приходится самостоятельно

### Паттерн разработки TDD

TDD - test-driven developement или разработка через тестирование. Достигается соблюдением трех правил.

**Три правила TDD**:

 - Продакшн-код можно писать только для починки падающего теста
 - В тесте нужно писать ровно столько кода, сколько необходимо чтобы он упал. Ошибки компиляции считаются падениями теста.
 - В прод можно написать ровно столько кода, сколько требуется для починки дного падающего теста.


 Получается следйющий пайплайн - пишем падающий тест, пишем код чтобы тест не падал, рефакторим код так, чтобы тесты не падали. Повторяем до сходимости.

1. [Test Driven Development: By Example 1st Edition](https://www.eecs.yorku.ca/course_archive/2003-04/W/3311/sectionM/case_studies/money/KentBeck_TDD_byexample.pdf)
2. [On Growing Object Oriented Software, Guided by Tests](https://www.amazon.com/Growing-Object-Oriented-Software-Guided-Tests/dp/0321503627)


### Каты

![](https://karate.by/uploads/posts/2010-10/kkk0001s.png)

Каты - упражнения по программированию, помогающие отточить навыки путем многократного повторнеия. Концепция взята из японских боевых искусств. Подробнее про них можно почитать в книжке [The Pragmatic Programmer](https://pragprog.com/titles/tpp20/the-pragmatic-programmer-20th-anniversary-edition/)



**Ката Greeter**

Данную кату надо выполнять строго по пунктам, не заглядывая вперед:

- Создайте класс `Greeter`, у которого есть метод `greet` принимающий на вход имя и возвращающий "Hello <имя>".
- Метод `greet` должен убирать лишние пробелы - в начале и в конце имени
- Метод `greet` должен возвращать ошибку если имя - пустая строка (или строка с пробелами)
- Метод `greet` возвращает "Good evening <имя>" если текущее время - 18:00-22:59

## Первый тест

Для автоматизированного тестирования написано много фреймворков на разных языках. Короткий список для python:

* [unittest](https://docs.python.org/3/library/unittest.html)
* [nose2](https://docs.nose2.io/en/latest/)
* [pytest](https://docs.pytest.org/en/latest/)

В рамках лекции мы остановимся на `pytest`. Почему он? Банально - он удобнее (если детально углубляться, глобально инчем не отличаются, просто в pytest уже достаточно много всего сделано из коробки)

In [ ]:
!pip install ipytest pytest coverage

In [2]:
import pytest
import ipytest
import coverage
ipytest.autoconfig()
__file__ = "testing_and_logging_lection.ipynb"

#### Как pytest находит тесты:

1. Рекурсивно находит все python-файлы в текущей директории
2. Оставляет только файлы вида `test_*.py` и `*_test.py`
3. В этих файлах
  1. Находит все функции с префиксом `test`
  2. Находит все методв с префиксом `test` внутри классов с префиксом `Test`. У классов не должно быть метода `__init__`
  
Поведение можно модифицировать. [Подробнее в документации](https://docs.pytest.org/en/stable/goodpractices.html#test-discovery)

Напишем минимальный тест (который, естественно, упадет)

In [3]:
%%ipytest -q
def test_greeter():
    Greeter()

F                                                                                            [100%]
============================================= FAILURES =============================================
___________________________________________ test_greeter ___________________________________________

    def test_greeter():
>       Greeter()
E       NameError: name 'Greeter' is not defined

<ipython-input-3-f0bf3d5080f2>:2: NameError
===================================== short test summary info ======================================
FAILED t_560cae9863044c19ac33e5104c62aee6.py::test_greeter - NameError: name 'Greeter' is not defined


`pytest` выводит отчет, в котором можно посмотреть сколько у нас всего тестов, какие из них упали и по какой причине.

Теперь сделаем так чтобы тест проходил

TDD выглядит следующим образом:

1. Написали тест

2. Написали кусок кода

3. Проверили, что проходит тест

4. PROFIT

In [8]:
class Greeter:
    pass

In [9]:
%%ipytest -q
def test_greeter():
    assert Greeter()

.                                                                                            [100%]


Сначала мы пишем тесты на реализацию что должно проверять, а только потом делаем реализацию.

Теперь наконец-то напишем нормальный тест, воспользовавшись основной фишкой `Pytest` - `assert`. `Pytest` находит все вызовы `assert` в коде тестов, а затем переписывает этот код так, чтобы в случае падения пользователь мог получить удобный дифф и трейсбек.

- [Демки разных ассертов](https://docs.pytest.org/en/stable/example/reportingdemo.html#tbreportdemo)
- [Цикл статей про то, как это работает](https://www.pythoninsight.com/2018/01/assertion-rewriting-in-pytest-part-1/)

Базово assert работает достаточно просто: мы вызываем некоторое выражение, которое должно выдавать True/False. Получилось True - мы молодцы, False - нет, роняем

In [10]:
%%ipytest -q
def test_greeter():
    name = "Mike"
    assert Greeter().greet(name) == name

F                                                                                            [100%]
============================================= FAILURES =============================================
___________________________________________ test_greeter ___________________________________________

    def test_greeter():
        name = "Mike"
>       assert Greeter().greet(name) == name
E       AttributeError: 'Greeter' object has no attribute 'greet'

<ipython-input-10-91756f0b423d>:3: AttributeError
===================================== short test summary info ======================================
FAILED t_560cae9863044c19ac33e5104c62aee6.py::test_greeter - AttributeError: 'Greeter' object has no attribute 'greet'


Еще одна итерация TDD:

In [12]:
class Greeter:
    def greet(self, name):
        return name

In [13]:
%%ipytest -q
def test_greeter():
    name = "Mike"
    assert Greeter().greet(name) == name

.                                                                                            [100%]


#  Параметризация

Наша реализация представляет собой немного не то что мы хотели. В чем проблема? А в том, что у нас написан всего 1 тест, который можно обработать изощренно:

In [14]:
class Greeter:
    def greet(self, name):
        return "Mike"

In [15]:
%%ipytest -q
def test_greeter():
    name = "Mike"
    assert Greeter().greet(name) == name

.                                                                                            [100%]


Наверное стоит добавить больше разных тестов, чтобы вводы были разные.

Чтобы не копировать один и тот же тест (и не плодить 100500 тестов), можно воспользоваться параметризацией:

Посмотрим что это такое на примере:

In [16]:
%%ipytest -q
import pytest


@pytest.mark.parametrize("a, b, c", [
    (1, 2, 3),
    (3, 4, 7),
    (5, 6, 11),
])
def test_sample_parametrization(a, b, c):
    assert a + b == c

...                                                                                          [100%]


Давайте рассмотрим что из себя оно представляет:

Конкректно у нас фигурирует тут: `pytest.mark.parametrize`
Декоратор, который позволяет параметризировать (использовать несколько наборов данных) на тестовую функцию.

Каждая комбинация значений будет автоматически передаваться в тестовую функцию и pytest выполнит для каждого набора тестовых данных.



```python
@pytest.mark.parametrize("<аргумент1>, <аргумент2>, <аргумент3>, ...", [<набор_Значений_для_аргумента_1>, <набор_значений_для_аргумента_2>, <набор_значений_для_аргумента_3>, ...]
ИЛИ [
    (<набор_Значений_для_аргумента_1>, <набор_значений_для_аргумента_2>, <набор_значений_для_аргумента_3>, ...),
    (<набор_Значений_для_аргумента_1>, <набор_значений_для_аргумента_2>, <набор_значений_для_аргумента_3>, ...),
    (<набор_Значений_для_аргумента_1>, <набор_значений_для_аргумента_2>, <набор_значений_для_аргумента_3>, ...)
])
def test_<название теста>(<аргумент1>, <аргумент2>, <аргумент3>, ...):

    <подготовка или демонстрация тестового сценария>

    assert <проверяемое выражение>

```


* аргументы в фикстурах -- переменные и для каждой переменной должно быть значение типа object (то есть любой объект)

Если не будет хоть одного значения тест завершится ошибкой с проверкой на количество аргументов и значений  `assert len(param.values) != len(argnames)`

In [17]:
%%ipytest -q

@pytest.mark.parametrize("a, b, c", [1, 2])
def test_sample(a, b, c):
    assert a + b == 3


============================================== ERRORS ==============================================
______________________ ERROR collecting t_560cae9863044c19ac33e5104c62aee6.py ______________________
/usr/local/lib/python3.10/dist-packages/pluggy/_hooks.py:513: in __call__
    return self._hookexec(self.name, self._hookimpls.copy(), kwargs, firstresult)
/usr/local/lib/python3.10/dist-packages/pluggy/_manager.py:120: in _hookexec
    return self._inner_hookexec(hook_name, methods, kwargs, firstresult)
/usr/local/lib/python3.10/dist-packages/_pytest/python.py:271: in pytest_pycollect_makeitem
    return list(collector._genfunctions(name, obj))
/usr/local/lib/python3.10/dist-packages/_pytest/python.py:498: in _genfunctions
    self.ihook.pytest_generate_tests.call_extra(methods, dict(metafunc=metafunc))
/usr/local/lib/python3.10/dist-packages/pluggy/_hooks.py:574: in call_extra
    return self._hookexec(self.name, hookimpls, kwargs, firstresult)
/usr/local/lib/python3.10/dist-packages/

Видим отчеты пайтеста во всей красе. Починим тесты:

In [18]:
%%ipytest -q
test_cases = [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")]

@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting

FFF                                                                                          [100%]
============================================= FAILURES =============================================
__________________________________ test_greeter[Mike-Hello Mike] ___________________________________

name = 'Mike', greeting = 'Hello Mike'

    @pytest.mark.parametrize("name, greeting", test_cases)
    def test_greeter(name, greeting):
>       assert Greeter().greet(name) == greeting
E       AssertionError: assert 'Mike' == 'Hello Mike'
E         - Hello Mike
E         + Mike

<ipython-input-18-e1e045608403>:5: AssertionError
__________________________________ test_greeter[John-Hello John] ___________________________________

name = 'John', greeting = 'Hello John'

    @pytest.mark.parametrize("name, greeting", test_cases)
    def test_greeter(name, greeting):
>       assert Greeter().greet(name) == greeting
E       AssertionError: assert 'Mike' == 'Hello John'
E         - Hello John
E

In [19]:
class Greeter:
    def greet(self, name):
        return "Hello " + name

In [20]:
%%ipytest -q
test_cases = [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")]

@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting

...                                                                                          [100%]


Ура, мы выполнили первый пункт!

Перейдем к следующему пункту нашего задания:

- Метод `greet` должен убирать лишние пробелы - в начале и в конце имени

Опять же, напишем тест:

In [21]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeting = greeter.greet(" Mike")
    assert not greeting.startswith(" ")

.                                                                                            [100%]


Обратим внимание что тест проходит и возникает соблазн продолжить работу. Однако если посмотреть на тест внимательно - можно увидеть в нем ошибку. (Какая?)


Чтобы не наступать на такие грабли существует **правило** - только что написанный тест должен падать, при чем именно из-за того поведения, которое этот тест должен было покрыть.


Вы можете писать тесты на уже существующий код - в таком случае они могут не падать т.к. код уже работает как надо. Тогда есть два варианта:
* Сделать в продовом коде баг чтобы тест упал
* Обратить проверяемое условие в тесте

Поправим наш тест:

In [22]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeted_name = greeter.greet(" Mike").split(" ", 1)[1]
    assert not greeted_name.startswith(" ")

F                                                                                            [100%]
============================================= FAILURES =============================================
___________________________________________ test_spaces ____________________________________________

    def test_spaces():
        greeter = Greeter()
        greeted_name = greeter.greet(" Mike").split(" ", 1)[1]
    
>       assert not greeted_name.startswith(" ")
E       AssertionError: assert not True
E        +  where True = <built-in method startswith of str object at 0x7ac7d11350f0>(' ')
E        +    where <built-in method startswith of str object at 0x7ac7d11350f0> = ' Mike'.startswith

<ipython-input-22-952847da0f49>:5: AssertionError
===================================== short test summary info ======================================
FAILED t_560cae9863044c19ac33e5104c62aee6.py::test_spaces - AssertionError: assert not True


In [23]:
class Greeter:
    def greet(self, name):
        if name.startswith(" "):
            name = name[1:]
        return "Hello " + name

In [24]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeted_name = greeter.greet(" Mike").split(" ", 1)[1]

    assert not greeted_name.startswith(" ")

.                                                                                            [100%]


Перечитаем наше задание:
* Метод greet должен убирать лишние пробелы - в начале и **в конце имени**

Видимо нам нужно расширить тест:

In [25]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeted_name = greeter.greet(" Mike ").split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

F                                                                                            [100%]
============================================= FAILURES =============================================
___________________________________________ test_spaces ____________________________________________

    def test_spaces():
        greeter = Greeter()
        greeted_name = greeter.greet(" Mike ").split(" ", 1)[1]
>       assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")
E       AssertionError: assert (not False and not True)
E        +  where False = <built-in method startswith of str object at 0x7ac7d145b2f0>(' ')
E        +    where <built-in method startswith of str object at 0x7ac7d145b2f0> = 'Mike '.startswith
E        +  and   True = <built-in method endswith of str object at 0x7ac7d145b2f0>(' ')
E        +    where <built-in method endswith of str object at 0x7ac7d145b2f0> = 'Mike '.endswith

<ipython-input-25-c19cda56cb96>:4: AssertionError
===========

Починим тест

In [26]:
class Greeter:
    def greet(self, name):
        if name.startswith(" "):
            name = name[1:]
        if name.endswith(" "):
            name = name[:1]
        return "Hello " + name

In [27]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeted_name = greeter.greet(" Mike ").split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

.                                                                                            [100%]


Наш тест все еще недостаточно хорош. Хороший набор тестов должен покрывать разные граничные условия и заходить во все ветки исполнения кода. Параметризуем наш тест так чтобы покрывал как можно ветвей исполнения кода:

In [28]:
%%ipytest -q

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "])
def test_spaces(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

....FF                                                                                       [100%]
============================================= FAILURES =============================================
_______________________________________ test_spaces[  Mike] ________________________________________

name = '  Mike'

    @pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "])
    def test_spaces(name):
        greeter = Greeter()
        greeted_name = greeter.greet(name).split(" ", 1)[1]
    
>       assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")
E       AssertionError: assert (not True)
E        +  where True = <built-in method startswith of str object at 0x7ac7d14ce6b0>(' ')
E        +    where <built-in method startswith of str object at 0x7ac7d14ce6b0> = ' Mike'.startswith

<ipython-input-28-06978f9470a6>:6: AssertionError
______________________________________ test_spaces[  Mike  ] _________________________________

Можно давать имена отдельным наборам параметров - тогда будет удобнее читать вывод пайтеста

In [29]:
%%ipytest -q

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

....FF                                                                                       [100%]
============================================= FAILURES =============================================
____________________________________ test_spaces[double space] _____________________________________

name = '  Mike'

    @pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                             ids=["no spaces", "left space", "right space",
                                  "two-side space", "double space", "two-sided double space"])
    def test_spaces(name):
        greeter = Greeter()
        greeted_name = greeter.greet(name).split(" ", 1)[1]
    
>       assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")
E       AssertionError: assert (not True)
E        +  where True = <built-in method startswith of str object at 0x7ac7d12c1470>(' ')
E        +    where <built-in method startswith of str object at 0x7ac7d12c1

In [30]:
class Greeter:
    def greet(self, name):
        while name.startswith(" "):
            name = name[1:]
        while name.endswith(" "):
            name = name[:1]
        return "Hello " + name

In [31]:
%%ipytest -q

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

......                                                                                       [100%]


Код кажется многословным! (как будто можем проще, правда?) После рефакторинга необходимо его подчистить!

In [32]:
class Greeter:
    def greet(self, name):
        return "Hello " + name.strip()

In [33]:
%%ipytest -q


@pytest.mark.parametrize("name, greeting", [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")])
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

.........                                                                                    [100%]


Шикарно, выполнили 2 пункта из 4, а также познакомились с параметризацией, можем сразу много кода написать

## Моки и фикстуры

Mock (с англ - подделка, заглушка и тд) -- объекты, которые создаются для замены реальных объектов в процессе тестирования.

Основная идея мока заключается в том, чтобы изолировать тестируемую часть системы от других компонентов, которые могут быть сложными, медленными или недоступными на момент тестирования.

Большая потребность их возникает тогда, когда мы пишем большое и сложное приложение в котором имеется очень сложная логика и невозможно протестировать as appropriate, уже сразу написанный код.

То есть логика следующая: мы не знаем, что у нас будет за объект/мы не хотим к нему обращаться. Чтобы протестировать функционал, нам необходимо создать какой-то фейк, который ***симулирует*** поведение нужного объекта. Вот это и есть Mock

Fixture (c англ - крепление, зацепка) -- уже подготовленные наборы данных, которые имеют состояние. В данном случае у нас уже есть какие-то данные, которые мы не хотим постоянно собирать (потому что собирать на каждый тест - долго/дорого). Идея: взять и зафиксировать их

Fixture != Mock

Рассмотрим типичную ситуацию когда прибегают к мокам:

**Проблема**
Есть класс который взаимодействует со внешней системой, количество запросов ограниченно. Нужно уметь тестировать логику в которой фигурирует вызов этой внешней системы независя от реальных вызовов.

In [34]:
class UserService:
    def __init__(self, db):
        self._db = db

    def get_user_status(self, user_id):
        user_data = self._db.get_user(user_id)  # представим что реализация и вызов этого метода нет
        if user_data.get('active'):
            return f"User {user_id} is active"
        return f"User {user_id} is deactivated"

Для того чтобы протестировать этот функционал приходят на помощь мокирование этого вызова. Мы пропишем в тесте что мы ожидаем от этого вызова и вернем значение:

In [ ]:
!pip install mock

In [37]:
%%ipytest -q

from mock import Mock

@pytest.mark.parametrize("user_id, db_response, expected_message", [
    (1, {"active": True}, f"User 1 is active"),   # Активный пользователь
    (2, {"active": False}, f"User 2 is deactivated"),  # Неактивный пользователь
    (3, {}, "User 3 is deactivated"),  # Пустые данные пользователя
])

def test_user_service(user_id, db_response, expected_message):
    mock_db = Mock()

    mock_db.get_user.return_value = db_response

    service = UserService(mock_db)

    result = service.get_user_status(user_id)
    assert result == expected_message

    mock_db.get_user.assert_called_once_with(user_id)

...                                                                                          [100%]


Давайте рассмотрим что мы описали тут:

* `mock_db = Mock()` -- создали пустой объект мока

* `mock_db.get_user.return_value` -- Перегрузка мока. В буквальном смысле: при вызове метода `get` объекта `db` верни ответ `db_response`.

Структура записи такова:

`<экземпляр класса мока>.<подменяющий вызывающий метод>.return_value = <значение>`


А что если мы хотим проверять вызов ошибки?

Давайте добавим в класс обработку ошибок:

In [38]:
class UserService:
    def __init__(self, db):
        self._db = db

    def get_user_status(self, user_id):
        try:
            user_data = self._db.get_user(user_id)
            if user_data.get('active'):
                return f"User {user_id} is active"
            return f"User {user_id} is deactivated"
        except Exception as e:
            return f"Error retrieving user {user_id}: {str(e)}"

Напишем сценарий в котором при вызове метода будет вызываться ошибка:

In [39]:
%%ipytest -q

@pytest.mark.parametrize("user_id, db_response, expected_message, should_raise", [
    (4, None, "Error retrieving user 4: Database connection failed", True),
])
def test_user_service(user_id, db_response, expected_message, should_raise):
    mock_db = Mock()

    if should_raise:
        mock_db.get_user.side_effect = Exception("Database connection failed")
    else:
        mock_db.get_user.return_value = db_response

    service = UserService(mock_db)
    result = service.get_user_status(user_id)
    assert result == expected_message
    assert mock_db.get_user.call_count == 1

.                                                                                            [100%]


Все что мы заготавливали мы оставляем все как и есть. Но только заменяем `return_value` на `side_effect` и прописываем какую ошибку мы ждали:

Структура записи такова:

`<экземпляр класса мока>.<подменяющий вызывающий метод>.side_effect = <значение>`

Наблюдаемая проблема:

* У нас появился Boiler Print в тестах от которого нужно избавляться!

Фикстуры так же могут чистить использованный инвентарь за создаваемым объектом в конце теста и иметь разный scope - например создваться на каждый тест, модуль или тред, запускающий тесты. [Подробнее в документации](https://docs.pytest.org/en/stable/fixture.html).

Scopes (скоупы) у фикстур бывают:
* function (по умолчанию)
* class -- внутри класса
* module -- создаваемый внутри модуля

Давайте перепишем это в фикстуру!

In [40]:
%%ipytest -q


@pytest.fixture(scope='function')
def mock_db(request):
    mock_db = Mock()

    db_response, should_raise = request.param

    if should_raise:
        mock_db.get_user.side_effect = Exception("Database connection failed")
    else:
        mock_db.get_user.return_value = db_response

    return mock_db


In [41]:
%%ipytest -q


@pytest.mark.parametrize("mock_db, user_id, expected_message", [
    (({"active": True}, False), 1, "User 1 is active"),
    (({"active": False}, False), 2, "User 2 is deactivated"),
    (({}, False), 3, "User 3 is deactivated"),
    ((None, True), 4, "Error retrieving user 4: Database connection failed"),
], indirect=["mock_db"])
def test_user_service(mock_db, user_id, expected_message):
    service = UserService(mock_db)

    result = service.get_user_status(user_id)

    assert result == expected_message

    assert mock_db.get_user.call_count == 1

....                                                                                         [100%]


Что мы тут сделали?
* создали фикстуру в которой происходит подготовка всего необходимого для возвращения значений
* немного поправили входные параметры в нашей параметризации: то есть мы прокинули туда Mock объект и прописали логику при которой будет вызываться новое поведение в `request.params`

Такой механизм называется "непрямая параметризация" - `indirect parametrization`

В `pytest` объект request автоматически передается в фикстуры, когда они вызываются.
Например, с его помощью можно получить параметры, переданные в фикстуру, или информацию о тестовой функции, которая используетcя оной.
Когда используется параметризация тестов с флагом `indirect`, параметры передаются через фикстуру. Внутри фикстуры они становятся доступными через `request.param`

Для БД фикстура может выглядеть примерно так:

In [42]:
%%ipytest -q

class DBConnection:
    pass

class TestDB:
    def init_db(self):
        print("init db")

    def get_connection(self):
        return DBConnection()

    def shutdown(self):
        print("close db")

@pytest.fixture(scope="module")
def db_connection():
    db = TestDB()
    db.init_db()
    try:
        yield db.get_connection()
    finally:
        db.shutdown()

def test_db_1(db_connection):
    assert db_connection

def test_db_2(db_connection):
    assert db_connection

..                                                                                           [100%]


Заключение:

* Мок может меняться и быть в тесте, а в свою очередь фикстура имеет свою структуру и не может меняться
* Моки легко параметризировать и писать различные тест кейсы на каждый случай
* Легко "вшивать фикстуры в тесты и прокидывать тестовые аргументы в параметры фикстуры тем самым задавая им свое поведение

#### Вернемся к решению каты:

Тесты нужно рефакторить. В основном главная задача тестов:
* проверять код, покрывая все возможные случаи на берегу (чтобы не отправить багу в production и при этом не потерять деньги)
* понятность и простота тестов (нельзя долго писать тесты на определенную фичу, это стоит очень дорого и ресурсы не предрасположены чтобы заниматься ими. Помните: тесты хоть и не помогают в зарабатыванию денег, но при этом без них есть шанс потерять много денег)
* documentation as a code (когда вы придете работать куда-нибудь, они вам помогут быстрее разобраться в коде, чем вы сами сидели и читали код)

**[Первая проблема]**:
* Имена тестов не очень информативны. Если упадет тест `test_greater` - будет не совсем понятно что именно тестировалось и что надо чинить. В целом имена тестам надо давать как можно более подробные - тесты вызываются автоматически, автоматике длинна имени безразлична, а вот человеку, читающему выхлоп пайтеста, лучше предоставить как можно больше информации.

[Статья на тему](https://enterprisecraftsmanship.com/posts/you-naming-tests-wrong)

In [43]:
%%ipytest -q

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

.........                                                                                    [100%]


**[Вторая проблема]**

В обоих тестах мы создаем `greeter`. Это привожит к дублированию кода. Кроме того, на практике вместо `greeter` у нас может быть какой-нибудь тяжелый объект типа базы даных или контекста сессии, который надо каждый раз инициализировать и чистить. Решить эти проблемы нам поможет механизм фикстур:

In [44]:
%%ipytest -q

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

.........                                                                                    [100%]


Так же в `pytest` есть разные встроенные фикстуры. [Список лежит здесь](https://docs.pytest.org/en/stable/fixture.html). Наиболее интересные:
* monkeypatch - временно можифицирует методы классов, модулей и т.д.
* testdir - создает верменную директорию для каждого теста, которую потом чистит

## Тестирование исключений, патчинг и работа с тестами где фигурируют флоты

Возвращаемся к катам. Третья часть:

- Метод `greet` должен возвращать ошибку если имя - пустая строка (или строка с пробелами)

Проблема: мы должны тестировать исключение, но при этом мы не должны его ловить стандартным try-catch в тесте а просто создать среду в котором будет **ожидаться** падение теста. Для этого необходимо это создать в тесте. На помощь приходит ```pytest.raises``` (который ожидает, что БУДЕТ ошибка)

In [45]:
%%ipytest -q

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

def test_greet_raises_value_error_on_empty_string(greeter):
    with pytest.raises(Exception): # (*)
        greeter.greet("")

# (*) обычно вы тут ждать определенную ошибку которую вы сами создали, к примеру вы написали класс ошибки которым отнаследовались от базового класса Exception

F                                                                                            [100%]
============================================= FAILURES =============================================
__________________________ test_greet_raises_value_error_on_empty_string ___________________________

greeter = <__main__.Greeter object at 0x7ac7d0427100>

    def test_greet_raises_value_error_on_empty_string(greeter):
>       with pytest.raises(Exception): # (*)
E       Failed: DID NOT RAISE <class 'Exception'>

<ipython-input-45-b4c7ee57e28b>:6: Failed
===================================== short test summary info ======================================
FAILED t_560cae9863044c19ac33e5104c62aee6.py::test_greet_raises_value_error_on_empty_string - Failed: DID NOT RAISE <class 'Exception'>


По тексту отчета видим, что тест ожидал исключения, но его не было. Починим тест:

In [63]:
class Greeter:
    def greet(self, name):
        name = name.strip()
        if not name:
            raise ValueError("Empty name!")
        return "Hello " + name

И дополнительно параметризуем:

In [53]:
%%ipytest -q

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

@pytest.mark.parametrize("name", ["", "   ", "  ", " "])
def test_greet_raises_value_error_on_empty_string(greeter, name):
    with pytest.raises(ValueError):
        greeter.greet(name)

.............                                                                                [100%]


Ура, осталась последняя ката:

- Метод `greet` возвращает "Good evening <имя>" если текущее время - 18:00-22:59

 ## Работа с тестами при участии времени

Пишем тест:

In [64]:
%%ipytest -q

import datetime

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

def test_greeting_is_good_evening_in_evening(monkeypatch, greeter):
    fake_time =  datetime.datetime(2020, 11, 10, 19)
    class mydatetime:
        @classmethod
        def now(cls):
            return fake_time

    monkeypatch.setattr(datetime, 'datetime', mydatetime)
    assert greeter.greet("Mike").startswith("Good evening")

F                                                                                            [100%]
============================================= FAILURES =============================================
_____________________________ test_greeting_is_good_evening_in_evening _____________________________

monkeypatch = <_pytest.monkeypatch.MonkeyPatch object at 0x7ac7d04d2c80>
greeter = <__main__.Greeter object at 0x7ac7d04d3a00>

    def test_greeting_is_good_evening_in_evening(monkeypatch, greeter):
        fake_time =  datetime.datetime(2020, 11, 10, 19)
        class mydatetime:
            @classmethod
            def now(cls):
                return fake_time
    
        monkeypatch.setattr(datetime, 'datetime', mydatetime)
>       assert greeter.greet("Mike").startswith("Good evening")
E       AssertionError: assert False
E        +  where False = <built-in method startswith of str object at 0x7ac7d0263cf0>('Good evening')
E        +    where <built-in method startswith of str obje

Изменим код, и разберем тест выше:

In [67]:
import datetime

class Greeter:
    def greet(self, name):
        name = name.strip()
        if not name:
            raise ValueError("Empty name!")
        hour = datetime.datetime.now().hour
        if 18 <= hour <= 22:
            return "Good evening " + name
        return "Hello " + name

Посмотрим, что со старыми тестами (они не будут падать, но как будто что-то не то)

In [68]:
%%ipytest -q

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

@pytest.mark.parametrize("name", ["", "   ", "  ", " "])
def test_greet_raises_value_error_on_empty_string(greeter, name):
    with pytest.raises(ValueError):
        greeter.greet(name)

.............                                                                                [100%]


Будет падать, потому что время текущее не матчится. Зададим дефолтное время.

In [70]:
%%ipytest -q
import datetime


@pytest.fixture(scope="function")
def set_time(monkeypatch):
    def set_time_(time):
        class mydatetime:
            @classmethod
            def now(cls):
                return time

        monkeypatch.setattr(datetime, 'datetime', mydatetime)
    yield set_time_


@pytest.fixture(scope="function")
def set_day_time(set_time):
    yield set_time(datetime.datetime(2020, 10, 10, 10))


@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(set_day_time, greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(set_day_time, greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")


def test_greeting_is_good_evening_in_evening(set_time, monkeypatch, greeter):
    set_time(datetime.datetime(2020, 11, 10, 19))
    assert greeter.greet("Mike").startswith("Good evening")

..........                                                                                   [100%]


Ну вроде все, у нас все удалось, поздравляю!

## Работа с плавающей точкой

Сравнение `float` сталось за кадром - разберем его отдельно.
 Из-за ошибок округления `float` трудно сравнивать через `==`

In [72]:
%%ipytest -q
def test_float():
    assert 0.1 + 0.2 == 0.3

F                                                                                            [100%]
============================================= FAILURES =============================================
____________________________________________ test_float ____________________________________________

    def test_float():
>       assert 0.1 + 0.2 == 0.3
E       assert (0.1 + 0.2) == 0.3

<ipython-input-72-d4fdc5effe72>:2: AssertionError
===================================== short test summary info ======================================
FAILED t_560cae9863044c19ac33e5104c62aee6.py::test_float - assert (0.1 + 0.2) == 0.3


На помощь приходит API pytest'а заиспользуем `pytest.approx`

In [71]:
%%ipytest -q
def test_float():
    assert [0.1 + 0.2, 0.5] == pytest.approx([0.3, 0.5])

.                                                                                            [100%]


## Манкипатчинг запросов

В вашей системе используется платное апи и просто так делать запросы дорого или невозможно. Но проверить логику необходимо.

Что для этого нужно?
* знать что приходит к нам на вход после сделанного запроса

Возьмем любую публичную API, к примеру апи которая предоставляет рандомные шутки

In [76]:
import requests

URL = "https://official-joke-api.appspot.com/random_joke"

print(requests.get(URL).text)

{"type":"programming","setup":"How many React developers does it take to change a lightbulb?","punchline":"None, they prefer dark mode.","id":410}


In [74]:
def get_json(url: str):
    """Takes a URL, and returns the JSON."""
    return requests.get(url).json()

Как мы могли заметить, структура JSON'а такова:

```json
{
    "type":"general",
    "setup":"Why didn't the skeleton go for prom?",
    "punchline":"Because it had nobody.",
    "id":396
}

```

In [77]:
%%ipytest -q
import pytest
import requests
import json

FAKED_ANSWER = {"type": "test0", "setup": "some test joke", "punchline": "some test punchline", "id": 0}

# делаем mock на  response-объект библиотеки requests
class MockResponse:
    @staticmethod
    def json():
        return FAKED_ANSWER


def test_get_json(monkeypatch):

    # Делаем фальшивый метод get
    def mock_get(*args, **kwargs):
        return MockResponse()


    monkeypatch.setattr(requests, "get", mock_get)

    result = get_json(URL)
    assert result["id"] == 0

.                                                                                            [100%]


Можно сделать также аналогично используя библиотеку `mock`

Подменим реальный запрос на фейковый результат и количество вызов просчета не будет. Также давай посчитаем сколько раз мы вызвали этот метод

In [78]:
%%ipytest -q
import mock
import requests

def test_with_mock_patch():
    mock_answer = mock.Mock()
    mock_answer.json.return_value = FAKED_ANSWER

    with mock.patch('requests.get', return_value=mock_answer) as mock_request:
        assert get_json(URL)['id'] == 0

    assert mock_request.call_count == 1

.                                                                                            [100%]


А что если у нас возврат метода это изменение чего нибудь? К примеру состояния чего-то там, или оно влияет на что-то и необходимо помимо возврата что-то менять...

In [79]:
class API:
    def compute(self, value):
        return value

api = API()

def process_data(value):
    return api.compute(value)

In [80]:
%%ipytest -q
def test_process_data():

    def side_effect_multiply_by_two(x):
        return x * 2

    mock_api = mock.Mock()
    mock_api.compute.side_effect = side_effect_multiply_by_two

    # Подменяем глобальный объект api на наш мок
    with mock.patch('__main__.api', mock_api):
        result = process_data(5)

    assert result == 10

.                                                                                            [100%]


`return_value` -- позволяет возвращать, то что должен возвращать определенный метод

`side_effect` -- позволяет заменить метод на свой метод который будет делать, то что написано в методе

## Итого

Мы сделали небольшую кату, познакомились с TDD и основной функциональностью `pytest`:

* Как пайтест находит тесты
* Ассерты `pyteset`
* Параметризация тестов
* Фикстуры
* Тестирование исключений
* Манкипатчинг

Какие в итоге профиты у тестов:
 - Тесты помогают следить за тем что код соответствует спецификации
 - Тесты позволяют рефакторить код и не бояться при этом посадить баг
 - Тесты документируют код

Что осталось за кадром:
 - Виды тестов - юнит, интеграционные и т.д.
 - Настройка тестов в ci/cd
 - Плагины pytest

## Логгирование

Импортнем и настроим всю его сущность для работы

In [81]:
import sys
import logging

logger = logging.getLogger('my_logger')

stdout_handler = logging.StreamHandler(stream=sys.stdout)
stdout_handler.setLevel(logging.INFO)
logger.addHandler(stdout_handler)

logger.warning('Watch out!')

Watch out!


**Логгирование и его уровни**

Логгировать можно при помощи следующих методов:
* `logger.debug()`
* `logger.info()`
* `logger.warning()`
* `logger.error()`
* `logger.critical()`
* `logger.exception()`


С самого маленького до самого большого:

|Уровень | Когда используется|
|:------ |:------------------|
|`DEBUG`|Для диагностической информации|
|`INFO`|Для подтверждения того, что всё работает как запланировано|
|`WARNING`|Когда нужно предупредить что вскоре возможна поломка или программа используется не совсем так как нужно|
|`ERROR`|Для логгирования сепъезных ошибок, из-за которых программа теряет часть функциональности|
|`CRITICAL`|Для логгирования ошибок, после которых программа не может продолжать работу|





Стандартный уровень логгирования - `WARNING`

In [82]:
logger.info('Will not be printed')
logger.warning('Will be printed')

Will be printed


Поменяем уровни логирования:

In [83]:
logger.setLevel(logging.INFO)
logger.info('Will not be printed')
logger.warning('Will be printed')
logger.setLevel(logging.WARNING)

Will not be printed


INFO:my_logger:Will not be printed


Will be printed


In [84]:
logger.log(logging.WARNING, 'Will be printed')

Will be printed


Также, существует такой вид как `logger.exception`, в нем есть особенность

In [86]:
try:
  1 / 0
except Exception as e: # будем ловить абсолютно любой экзепшен и словим
  logger.exception("Caught error:")

Caught error:
Traceback (most recent call last):
  File "<ipython-input-86-27946ed9687e>", line 2, in <cell line: 1>
    1 / 0
ZeroDivisionError: division by zero


ERROR:my_logger:Caught error:
Traceback (most recent call last):
  File "<ipython-input-86-27946ed9687e>", line 2, in <cell line: 1>
    1 / 0
ZeroDivisionError: division by zero


In [87]:
def process_data(data):
    logger.debug(f"Received data: {data}")

    if not data:
        logger.error("No data received!")
        return None

    logger.info("Processing data...")
    processed_data = data.upper()

    logger.info(f"Data processed: {processed_data}")
    return processed_data

process_data("example data")

'EXAMPLE DATA'

In [ ]:
process_data("")

No data received!


# Логгирование в файл и хендлеры

Научимся логгировать в файл (напоминание с прошлого года)

In [88]:
fh = logging.FileHandler('debug.log')
fh.setLevel(logging.DEBUG)  # Выставляем уровень сообщений, которые будут логгироваться в файл.
logger.addHandler(fh)

In [89]:
logger.setLevel(logging.DEBUG)
logger.debug("Debug message")  # Не попадет в stdout, зато попадет в файл
print ("debug.log contents:")

with open("debug.log") as f:
    for l in f.readlines():
        print(l)

DEBUG:my_logger:Debug message


debug.log contents:
Debug message



Другие полезные хендлеры из библиотеки `logging`:

* `StreamHandler` - используется для логгирования в `stderr` и `stdout`
* `RotatingFileHandler` - Работает как файл хендлер, но при этом если файл, в который пишет логгер, достигнет определенного размера, начнет писать в новый файл. Старый файл либо удалит, либо оставит как бекап. Число бэкапов настраивается.
* `TimedRotatingFileHandler` - Работает как логгер выше, но файлы делятся не по размеру, а по времени записей
* `NullHandler` - Используется чтобы заглушить какой-нибудь логгер

In [ ]:
# StreamHandler

stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setLevel(logging.INFO)
stream_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(stream_handler)

In [ ]:
logger.info("Инфо в stdout")

2024-09-22 04:39:03,560 - INFO - Инфо в stdout


In [ ]:
logger.error("Ошибка!")

2024-09-22 04:39:05,313 - ERROR - Ошибка!


RotatingFileHandler используется для создания файлов логов с ограничением по размеру.

Когда файл достигает указанного размера, создается новый файл, а старый либо удаляется, либо сохраняется в виде бэкапа.

In [ ]:
# RotatingFileHandler

import logging
from logging.handlers import RotatingFileHandler

rotating_handler = RotatingFileHandler('app.log', maxBytes=2000, backupCount=5)
rotating_handler.setLevel(logging.DEBUG)
rotating_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(rotating_handler)

for i in range(300):
    logger.debug(f"Номер лога {i}")

TimedRotatingFileHandler логгирование с ротацией файлы логов не по размеру, а по времени.

Например, можно настроить ротацию каждый день, час, или минуту.

In [ ]:
#TimedRotatingFileHandler

import logging
from logging.handlers import TimedRotatingFileHandler

timed_handler = TimedRotatingFileHandler('timed_app.log', when='midnight', interval=1, backupCount=7)
timed_handler.setLevel(logging.INFO)
timed_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(timed_handler)

logger.info("Эта запись в журнале будет обновляться каждую полночь....")

2024-09-22 04:43:40,064 - INFO - This log entry will rotate every midnight.


NullHandler: Отключение логирования

Например у нас много логгеров и нам нужно отключить какой то из них...

In [ ]:
import logging

null_handler = logging.NullHandler()

logger = logging.getLogger(__name__)
logger.addHandler(null_handler)

logger.info("Вырубили логгер")

2024-09-22 04:43:35,118 - INFO - Вырубили логгер


## Попугай дня

![](https://do-slez.com/uploads/posts/2020-02/1582813051_pesquets-dracula-parrots-birds-new-guinea-1-5e55392f17e1e__700.jpg)

Сегодня у нас очень красивый орлиный попугай (орлиный за счет его клюва) и сходит в семейство щетиноголовых попугаев (видите какая щетина у него прям)

![](https://img.theepochtimes.com/assets/uploads/2020/05/14/Dracula-Parrot-i.jpg)

Исторически живет в Новой Гвинее и его очень редко можно встретить в зоопарках из-за очень прихотливого питания (им обязательно нужны тропические фрукты для ферментации) и требований к содержанию (температура, влажность)

Еще один, к сожалению, вымирающий вид, потому что на них охотятся индейцы за их красные перья (хоть и достаточно распространен в авикультуре, но выращивать его очень сложно).